# Multi-Agent Systems with Semantic Kernel

This notebook demonstrates how to create a multi-agent conversation system using Semantic Kernel's AgentGroupChat. We'll build a travel recommendation system where multiple AI agents collaborate - a Front Desk agent provides recommendations while a Concierge agent reviews and approves them. The system uses intelligent selection and termination strategies to manage the conversation flow.

## Import the Needed Packages

We'll import all the necessary libraries for creating a multi-agent system, including Semantic Kernel's agent framework, conversation strategies, and Azure OpenAI integration.

In [1]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential

from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import KernelFunctionFromPrompt

## Kernel Setup Function

Create a helper function to set up the Semantic Kernel with Azure OpenAI connection. This kernel will be used by all agents in our multi-agent system.

In [2]:
def _create_kernel_with_chat_completion() -> Kernel:
    load_dotenv()
    
    kernel = Kernel()
    # Option 1: Using API Key (recommended for development)
    chat_completion_service = AzureChatCompletion(
        deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
        endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY")
    )

    # Option 2: Using Azure AD Authentication (uncomment to use)
    # Create Azure credential 
    credential = DefaultAzureCredential()

    # Create a token provider function
    def get_azure_ad_token():
        """Function to get Azure AD token for OpenAI."""
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        return token.token

    # chat_completion_service = AzureChatCompletion(
    #     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    #     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    #     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    #     ad_token=get_azure_ad_token()

    kernel.add_service(chat_completion_service)

    return kernel

## Multi-Agent Travel Recommendation System

Create a sophisticated multi-agent conversation system where two specialized agents collaborate to provide travel recommendations. The system includes intelligent conversation management with custom selection and termination strategies.

In [3]:
async def main():
    REVIEWER_NAME = "Concierge"
    REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """
    agent_reviewer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion(),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    FRONTDESK_NAME = "FrontDesk"
    FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """
    agent_writer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion(),
        name=FRONTDESK_NAME,
        instructions=FRONTDESK_INSTRUCTIONS,
    )

    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt="""
        Determine if the recommendation process is complete.
        
        The process is complete when the Concierge provides approval for any recommendation made by the Front Desk.
        Look for phrases like "approved", "this recommendation is approved", or any clear indication that the Concierge is satisfied with the suggestion.
        
        If the Concierge has given approval in their most recent response, respond with: yes
        Otherwise, respond with: no
        
        History:
        {{$history}}
        """,
    )

    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
        Determine which participant takes the next turn in a conversation based on the the most recent participant.
        State only the name of the participant to take the next turn.
        No participant should take more than one turn in a row.
        
        Choose only from these participants:
        - {REVIEWER_NAME}
        - {FRONTDESK_NAME}
        
        Always follow these rules when selecting the next participant, each conversation should be at least 4 turns:
        - After user input, it is {FRONTDESK_NAME}'s turn.
        - After {FRONTDESK_NAME} replies, it is {REVIEWER_NAME}'s turn.
        - After {REVIEWER_NAME} provides feedback, it is {FRONTDESK_NAME}'s turn.

        History:
        {{{{$history}}}}
        """,
    )

    chat = AgentGroupChat(
        agents=[agent_writer, agent_reviewer],
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer],
            function=termination_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]).lower() == "yes",
            history_variable_name="history",
            maximum_iterations=10,
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(
                result.value[0]) if result.value is not None else FRONTDESK_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    user_input = "I would like to go to Paris."

    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
    print(f"# User: '{user_input}'")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {chat.is_complete}")

await main()

# User: 'I would like to go to Paris.'
# Agent - FrontDesk: 'Visit the Louvre Museum to see world-famous art like the Mona Lisa.'
# Agent - Concierge: 'This suggestion is incredibly touristy and doesn’t quite align with what I envision as an authentic experience. While the Louvre is undoubtedly a cultural treasure, an alternative approach could be to explore lesser-known art spaces, hidden gems, or local artists thriving outside the tourist clusters. Seek out creative expressions that offer insight into Paris’s art scene today, rather than simply following the well-trodden path.'
# Agent - FrontDesk: 'Take a sunset stroll at Sacré-Cœur in Montmartre for stunning views of the city.'
# Agent - Concierge: 'This recommendation, while charming, is still a fairly popular and tourist-heavy option. To refine it, consider suggesting a walk through Montmartre’s lesser-frequented streets early in the morning or a tucked-away café in the area frequented by locals. This will capture the bohemian sp

## Experiment with Different Travel Requests

Try modifying the user input in the code above to test how the multi-agent system handles different travel scenarios. The agents will collaborate to provide and refine recommendations based on your input.

In [ ]:
# Example: Try different travel requests
# Modify the user_input variable in the main() function above with these examples:

example_requests = [
    "I would like to explore authentic local cuisine in Tokyo.",
    "I want to experience the real culture of Barcelona, not tourist traps.",
    "I'm looking for hidden gems in Rome that locals actually visit.",
    "I need recommendations for off-the-beaten-path activities in New York City.",
    "I want to discover authentic experiences in Prague that most tourists miss.",
    "I'm interested in local artisan workshops and markets in Istanbul."
]

# Replace the user_input in the main() function with any of these examples
# Then run the main() function again to see how the agents collaborate

print("Try these example requests by modifying the user_input in the main() function above:")
for i, request in enumerate(example_requests, 1):
    print(f"{i}. {request}")
    
print("\nWatch how the Front Desk agent provides recommendations and the Concierge agent reviews them!")
print("The conversation continues until the Concierge approves a recommendation.")

## Understanding the Multi-Agent Architecture

This system demonstrates several advanced concepts in multi-agent AI systems:

### **Agent Roles and Specialization**

1. **Front Desk Agent**: 
   - Specializes in providing travel recommendations
   - Designed to be brief and focused on practical suggestions
   - Responds to user requests and feedback from the Concierge

2. **Concierge Agent**: 
   - Acts as a quality reviewer focused on authentic, local experiences
   - Evaluates recommendations for their non-touristy value
   - Provides feedback to improve suggestions or approves final recommendations

### **Conversation Management Strategies**

1. **Selection Strategy (`KernelFunctionSelectionStrategy`)**:
   - Determines which agent speaks next based on conversation context
   - Ensures proper turn-taking between agents
   - Prevents agents from dominating the conversation

2. **Termination Strategy (`KernelFunctionTerminationStrategy`)**:
   - Monitors the conversation for completion conditions
   - Ends the conversation when the Concierge approves a recommendation
   - Includes safety limits (maximum iterations) to prevent infinite loops

### **Key Benefits of This Approach**

- **Specialized Expertise**: Each agent has a focused role and expertise
- **Quality Control**: The review process ensures better recommendations
- **Natural Conversation Flow**: Agents collaborate like human experts would
- **Flexible and Extensible**: Easy to add new agents or modify behaviors

## Expected Conversation Flow

When you run the system, you should see a conversation pattern like this:

```
# User: 'I would like to go to Paris.'
# Agent - FrontDesk: 'I recommend visiting the Marché aux Puces de Saint-Ouen, Paris's famous flea market...'
# Agent - Concierge: 'This recommendation shows good local insight, but could be refined...'
# Agent - FrontDesk: 'Consider exploring the covered passages like Galerie Vivienne...'
# Agent - Concierge: 'This recommendation is approved. The covered passages offer authentic Parisian experiences...'
# IS COMPLETE: True
```

### **What to Observe:**

1. **Turn-taking**: Agents alternate speaking based on the selection strategy
2. **Refinement Process**: The Concierge provides feedback to improve recommendations
3. **Completion**: The conversation ends when the Concierge approves a suggestion
4. **Quality Focus**: Both agents work toward authentic, non-touristy experiences

### **Conversation Length:**

- Minimum 4 turns to ensure proper collaboration
- Maximum 10 iterations to prevent infinite loops
- Natural completion when approval is given

## Customization Ideas

You can extend this multi-agent system in various ways:

### **Add More Agents**
```python
# Example: Add a Budget Advisor agent
BUDGET_NAME = "BudgetAdvisor"
BUDGET_INSTRUCTIONS = """
You are a budget-conscious travel advisor who evaluates recommendations for cost-effectiveness.
Provide feedback on whether recommendations are worth the cost and suggest budget alternatives.
"""
```

### **Modify Agent Personalities**
- Change the Front Desk agent to be more detailed or casual
- Make the Concierge focus on different aspects (budget, safety, adventure level)
- Add cultural expertise or language considerations

### **Adjust Conversation Rules**
- Change the minimum/maximum conversation length
- Modify termination conditions (e.g., require multiple approvals)
- Add new selection patterns (round-robin, expertise-based)

### **Domain Adaptation**
This pattern works for many collaborative scenarios:
- **Code Review**: Developer + Senior Developer + Security Expert
- **Content Creation**: Writer + Editor + Subject Matter Expert  
- **Product Planning**: Product Manager + Engineer + Designer
- **Medical Consultation**: General Practitioner + Specialist + Pharmacist

Try adapting the agents and instructions for different domains!